Imported Libaries

In [ ]:
import os
import re

cwd = 'C:\Users\jf79\Repos\FOI-Topic-Modelling'
import myfuncs.myfuncs as mf
import myfuncs.repofuncs as rf
import myfuncs.plotfuncs as pf

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

Script Specifc Imported Libaries

In [ ]:
from tqdm import tqdm
import itertools

import threading
from concurrent.futures import ThreadPoolExecutor

from queue import Queue
results_queue = Queue()



import nltk
nltk_data_path = os.getenv('', 'default/path')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import spacy
from spellchecker import SpellChecker
from wordsegment import Segmenter

# Initialize external tools
spell = SpellChecker()
custom_segmenter = Segmenter()
custom_segmenter.load()  # Load data for wordsegment
lemmatizer = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

vocabulary = [
    'FOI','vapes','vape','riley'
]
custom_stopwords = {
    'freedom','information','act','following','report',
    'please','dear','yes','request','provide',
    'hammersmith','fulham','like','email','requesting',
    'see','attach','foi','attached','service','would','many',
    'faithfully','number','council','authority','local','year',
    'borough','name','could','data','within','question','including',
    'copy','also','made','regarding','since','relating','requested',
    'thank','confirm','sent','make','date','much','kind','regard',
    'good','afternoon','evening','sir','madam','hello','per','correspondence',
    'address','total','contact','look','forward','hearing','way','day','apologise',
    'responding','grateful','necessary','period','list','time','related','detail',
    'use','end','last'
}
month_stopwords = {
    'january','february','march','april',
    'may','june','july','august','september',
    'october','november','december','jan','feb',
    'mar','apr','may','jun','jul','aug','sep',
    'oct','nov','dec','sept'
}

spell.word_frequency.load_words(vocabulary)
for word in vocabulary:
    custom_segmenter.unigrams[word] = 1e9 # Ensuring words in the vocabulary are being recognised in the Segmenter
stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)
stop_words.update(month_stopwords)

import gensim
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

import pyLDAvis.gensim_models as gensimvis
import pickle
import pyLDAvis

In [ ]:
# Read the CSV file into a DataFrame
cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\FOI Topic Modelling\Data'
os.chdir(cwd)
mf.read_directory()
columns = [
    'ClosedIRKey','CaseType','Division','Service','DateReceived','DateClosed','Due',
    'Status','Directorate','Team','Currentstage','Outcome','Outcomedate','ContactMethod',
    'Rating','DateAccepted','AssignedTo','Title','FirstName','LastName','Name','Address',
    'Town','County','Postcode','Details','TimescaleExtensionReason','ExtensionReason',
    'Extensionpermittedbylegislation','Representative'
]
FOI_details_data = pd.read_csv('FOI Details Data.csv', usecols=columns)

In [ ]:
FOI_details = FOI_details_data.copy()

# Data transformations
columns = [
    'Title','FirstName','LastName','Name',
    'Address','Town','County','Postcode'
]
FOI_address_details = pd.concat(
    (FOI_details[['ClosedIRKey']], FOI_details[columns]),
    axis=1
)
FOI_details = FOI_details.drop(columns=columns)
FOI_details['Details'] = FOI_details['Details'].fillna('').astype(str)
FOI_details.rename(columns={'Details':'details'}, inplace=True)

In [ ]:
# Data Transformation for Topic Modelling
FOI_request = FOI_details[['details']].copy()

# Apply processing to 'details' and 'summary' columns
FOI_request['details'] = FOI_request['details'].apply(rf.process_text)

# Tokenize 'details' and 'summary' columns
FOI_request['tokens'] = pd.DataFrame(FOI_request['details'].apply(rf.safe_tokenize))

mf.export_to_csv()

In [ ]:
# Read the CSV file into a DataFrame
cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\FOI Sentiment Analysis\Data'
os.chdir(cwd)
mf.read_directory()

processed_fois = pd.read_csv('processed_fois.csv')
FOI_email_data = pd.read_excel('FOI Details Data (2024).xlsx')
email_lookup_data = pd.read_excel('Email Domain Lookup.xlsx')

In [ ]:
FOI_email = FOI_email_data.copy()
FOI_email = email_lookup_data.copy()


processed_fois.columns = processed_fois.columns.str.lower().str.strip().str.replace(' ','_')
FOI_email.columns = FOI_email.columns.str.lower().str.strip().str.replace(' ','_')
FOI_email.rename(columns={'caseid':'caseid_final'},inplace=True)
processed_fois = processed_fois[processed_fois['processed_details'] != '']
processed_fois['processed_tokens'] = processed_fois['processed_details'].apply(rf.safe_tokenize)
processed_fois['new_tokens'] = processed_fois['processed_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
processed_fois = pd.merge(
    processed_fois, FOI_email,
    how='left', left_on='caseid', right_on='caseid_final',
    suffixes=('', '_drop')
)
processed_fois.drop([col for col in processed_fois.columns if '_drop' in col], axis=1, inplace=True)
# count_words(processed_fois['new_tokens'])
processed_fois = processed_fois[processed_fois['caseid_final'].isna() == False]
processed_fois['email'] = processed_fois['email'].fillna('')

# Apply the function to the email column
processed_fois['domain'] = processed_fois['email'].apply(rf.extract_domain)
processed_fois[['group','domain']] = processed_fois['domain'].str.split('.',n=1, expand=True)
columns_to_drop = ['processed_tokens','new_tokens']
columns = [x for x in processed_fois.columns.to_list() if x not in columns_to_drop]
processed_fois = pd.merge(
    processed_fois, FOI_email,
    how='left', left_on='group', right_on='domain',
    suffixes=('', '_drop')
)
processed_fois['category'] = processed_fois['category'].fillna('Unknown') 
mf.export_to_csv(processed_fois)

In [ ]:
import plotly.graph_objects as go
 
# Count the occurrences of each unique pair
counts = processed_fois.groupby(['category', 'directorate']).size().reset_index(name='Value')
 
# Create a mapping of unique labels to indices
unique_labels = list(set(counts['category'].tolist() + counts['directorate'].tolist()))
label_to_index = {label: i for i, label in enumerate(unique_labels)}
 
# Map category and directorate to their indices
counts['categoryIndex'] = counts['category'].map(label_to_index)
counts['directorateIndex'] = counts['directorate'].map(label_to_index)
 
# Prepare data for Sankey diagram
category = counts['categoryIndex'].tolist()
directorate = counts['directorateIndex'].tolist()
value = counts['Value'].tolist()
 
# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=7,
        thickness=20,
        line=dict(color="black", width=0.01),
        label=unique_labels
    ),
    link=dict(
        source=category,
        target=directorate,
        value=value
    )
)])
 
fig.update_layout(
    title_text="Sankey Diagram", font_size=10,
    width=1000,
    height=800    
)
fig.show()

fig.write_html('sankey_diagram.html')

In [ ]:
FOI_corpus = rf.prepare_corpus(FOI_final['new_tokens'],threshold=50)

# Create Dictionary
id2words = Dictionary(FOI_corpus)

# Term Document Frequency
corpus = [id2words.doc2bow(text) for text in FOI_corpus]

In [ ]:
# Initialize variables
results = pd.DataFrame(columns=['Validation_Set', 'Topics', 'Alpha', 'Beta', 'Coherence'])
lock = threading.Lock()  # Thread-safe lock

# Number of documents in the corpus
num_of_docs = len(corpus)

# Prepare corpus sets (75% and 100% corpus)
corpus_sets = [
    gensim.utils.ClippedCorpus(corpus, int(num_of_docs * 0.75)),
    corpus
]

# Hyper-parameters
min_topics = 2
max_topics = 15
step_size = 1
topics_range = list(range(min_topics,max_topics+1,step_size))
alphas = np.round(np.linspace(0.01, 0.91, 5), 2).tolist() + ['symmetric', 'asymmetric']
betas = np.round(np.linspace(0.01, 0.91, 5), 2).tolist() + ['symmetric']
corpus_titles = ['75% Corpus', '100% Corpus']

# Define test combinations to run in parallel
test_combinations = [
    (rf.find_element(corpus_title),topic,alpha,beta,corpus_title)
    for topic,alpha,beta,corpus_title, in itertools.product(topics_range,alphas,betas,corpus_titles)
]

tasks = [lambda combination=combination: rf.compute_for_params(*combination) for combination in test_combinations]
# Process tasks in parallel with a progress bar
with tqdm(total=len(test_combinations), desc='Processing combinations') as pbar:
    def task_with_progress(task):
        task()
        pbar.update(1)
    
    rf.run_io_task_in_parallel([lambda task=task: task_with_progress(task) for task in tasks])

while not results_queue.empty():
    result = results_queue.get()
    results = pd.concat([results, pd.DataFrame([result])], ignore_index=True)

display(results)
mf.export_to_csv(results)

In [ ]:
num_topics = 6
# Build LDA model
lda_model = gensim.models.LdaMulticore(
    corpus=corpus,
    id2word=id2words,
    num_topics=num_topics,
    random_state=42,
    chunksize=100,
    passes=10,
    alpha=0.24,
    eta=0.91
)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=FOI_corpus, dictionary=id2words, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)
# Assuming lda_model is your trained LdaModel
lda_model.save(f"C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/FOI Sentiment Analysis/Models/lda_model ({num_topics})")

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
path = 'C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/FOI Sentiment Analysis/Data'
LDAvis_data_filepath = os.path.join(path+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2words)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared,path+str(num_topics)+'.html')

LDAvis_prepared

In [ ]:
lambd = 0.2  # A specific relevance metric value
 
all_topics = {}
all_topics_with_docs = {}
num_topics = lda_model.num_topics
num_terms = 30
num_docs = 5  # Number of most related documents to retrieve for each topic
 
topic_doc_matrix = lda_model.get_document_topics(corpus, minimum_probability=0)  # Get the topic-document distribution
print
for i in range(1, num_topics + 1):  # Indices are 1-based, not 0-based
    # Extract the term information
    topic = LDAvis_prepared.topic_info[LDAvis_prepared.topic_info.Category == 'Topic' + str(i)].copy()
    topic['relevance'] = topic['loglift'] * (1 - lambd) + topic['logprob'] * lambd
    all_topics['Topic ' + str(i)] = topic.sort_values(by='relevance', ascending=False).Term[:num_terms].values
 
    # Find the most related documents for the topic
    topic_index = i - 1  # Convert to 0-based index
    doc_relevance = [doc[topic_index][1] for doc in topic_doc_matrix]  # Extract the relevance of documents for this topic
    top_doc_indices = sorted(range(len(doc_relevance)), key=lambda x: doc_relevance[x], reverse=True)
    sorted_doc_relevance = [doc_relevance[i] for i in top_doc_indices]
    
    # Retrieve the actual document texts
    top_docs_content = [FOI_final['details'][idx] for idx in top_doc_indices]  # Assuming `documents` contains the original texts
    all_topics_with_docs['Topic ' + str(i)] = {
        'terms': all_topics['Topic ' + str(i)],
        'top_docs_indices': top_doc_indices,  # Indice of most relevant doc
        'top_doc_relevance': sorted_doc_relevance,  # relevance of doc
        # 'top_docs': top_docs_content  # Contents of the most relevant documents
    }
 
# Create a DataFrame to display terms and documents for each topic
output_data = {
    'Topic': [],
    'Top Terms': [],
    'Top Doc Indices': [],
    'Top Doc Relevance': [],
    # 'Top Documents': []
}

for topic, data in all_topics_with_docs.items():
    output_data['Topic'].append(topic)
    output_data['Top Terms'].append(", ".join(data['terms']))
    output_data['Top Doc Indices'].append(", ".join(map(str,data['top_docs_indices'])))
    output_data['Top Doc Relevance'].append(", ".join(map(str,data['top_doc_relevance'])))
    # output_data['Top Documents'].append(" || ".join(data['top_docs']))  # Separate documents with '||'

output_data = pd.DataFrame(output_data)
mf.export_to_csv(output_data)

In [ ]:
output = output_data.copy()
FOI_topics = FOI_final.copy()
# Step 1: Convert 'col1' and 'col2' into lists
output['Top Doc Indices List'] = output['Top Doc Indices'].str.split(',')
output['Top Doc Relevance List'] = output['Top Doc Relevance'].str.split(',')

# Step 2: Create a new column that combines both lists into a dictionary
output['combined_dict'] = output.apply(lambda row: dict(zip(row['Top Doc Indices List'], row['Top Doc Relevance List'])), axis=1)
output.drop(['Top Doc Indices List','Top Doc Relevance List','Top Doc Indices','Top Doc Relevance', 'Top Terms'],axis=1,inplace=True)
output.set_index('Topic',inplace=True)

expanded_rows = output['combined_dict'].apply(pd.Series).stack().reset_index()
expanded_rows['index'] = expanded_rows['level_1'].astype('Int64')

columns = [
    'caseid',
    'directorate',
    'category'
]
FOI_topics = FOI_topics[columns]
FOI_topics.reset_index(inplace=True)
FOI_merged = pd.merge(
    expanded_rows, FOI_topics,
    how='left', on='index',
    suffixes=('', '_drop')
)
FOI_merged.drop(columns=['level_1','index'],inplace=True)
FOI_merged.rename(columns={0:'Relevance'},inplace=True)
FOI_merged['Relevance'] = FOI_merged['Relevance'].astype(float)

topic_names = {
    'Topic 1' : 'Housing and Support',
    'Topic 2' : 'Contract Management',
    'Topic 3' : 'Education and Schools',
    'Topic 4' : 'Legal and Regulatory',
    'Topic 5' : 'Retail and Consumer Issues',
    'Topic 6' : 'Property and Business'
}
FOI_merged['Topic'] = FOI_merged['Topic'].map(topic_names)
FOI_merged
pf.sankey3_diagram(FOI_merged,source='category',intermediate='Topic',target='directorate', value='Relevance', name='DIRECTORATE')

In [ ]:
topic_doc_matrix = lda_model.get_document_topics(corpus, minimum_probability=0)  # Get the topic-document distribution
for i in range(1, num_topics + 1):  # Indices are 1-based, not 0-based
    # Find the most related documents for the topic
    topic_index = i - 1  # Convert to 0-based index
    doc_relevance = [doc[topic_index][1] for doc in topic_doc_matrix]  # Extract the relevance of documents for this topic
    top_doc_indices = sorted(range(len(doc_relevance)), key=lambda x: doc_relevance[x], reverse=True)[:num_docs]
    print(doc_relevance[0])
    print(f'{topic_index}: {topic_doc_matrix[0][topic_index][1]}')

In [ ]:
doc_relevance = [doc[topic_index][1] for doc in topic_doc_matrix]
topic_doc_matrix[0][0][1]